---
# this is YAML front matter
layout: post
title: Anatomy of a Python Flask Web Application
description: A discussion of key elements in a Python Flask backend project.  This includes preparing a project for deployment.
courses: { csp: {week: 4, categories: [6.B]} }
categories: [C4.0]
type: devops
---

## Overview of Flask
This article overviews key considerations in setting up a Python Flask backend project.

## Overview of Python files to support a Flask server
> Discuss how to develop a home page, code, run local server test, and then sync to deploy to GitHub Pages.
- Review tools setup and make in README.md to support this lesson.  

## Files and Directories in this Project
> These are some of the key files and directories in this project
- ```README.md``` file contains instruction to setup tools and clone the project.  A README is a standard for all properly setup GitHub project.
- ```requirements.txt``` file contains the dependencies required to make this Python project into a Flask/Python project.  Additionally, other backend requirements are contained within ths file, for example dependencies for making a database.
- ```main.py``` is a Python source file to run the project.  This file starts a Flask web server locally, on localhost.  This is the file you start during development to run, test, and debug the project.
- ```Dockerfile``` and ```docker-compose.yml``` are the official files to run and test the project.  These files run the project in the same manner as when it is deployed on a server.  By running this file you check to see if your tools and dependencies will work on another machine, ie AWS EC2 server.
- ```instances``` this is the standard location for putting data files that you want to remain on server.  For instance, SQLite database files are stored in this directory.
- ```static``` directory is the standard location for files that you want to have cached by the web server.  For instance image files (jpeg, png, ...) or js files that remain constant during execution of the web server.
- ```api``` is the directory that contains code to receive and respond to a request from an external server.  This is the interface from the external world into logic and code in the remainder of this project.
- ```model``` these are the files that contain backend implementation for many of the files in the api directory.  For instance, there are files in model which directly interact with the database.
- ```templates``` these are files and directories that are used to support home and error pages for this web site.   
- ```.gitignore``` this file excludes elements from version controls.  We exclde files when they are built, they are NOT original source but derived.  You will see some files as dim in the VSCode Explorer, these files are purposely excluded from version control by rules in .gitignore.

* Note, there are many other key files and directories.  But, those files will be highlighted as we progress in development.


### Dockerfile and docker-compose.yml
> Make sure your docker files are created using these examples.  These files are the best way to determine if your web application is ready for deployment.

Examining and understanding these files will help you understand a great deal about your project.

- Dockerfile 
    - python 3.10 environment is used as or deployment container
    - python3-pip is installed, as pip is the package manager
    - requirement.txt is used to update project dependencies in the container
    - gunicorn is used to run a production version of python/flask, in this configuration 3 workers or 3 instances of the server are run.  This balances load.
    - main:app means the main.py is the primary file and app contains server configuration
    
- docker-compose.yml
    - image name the docker container
    - port declares address for web application on web server
    - volumes declares that the instance directory contains persistant data
    - restart tells web applciation to come back up if something like power outage or reboot takes down web server


In [ ]:
FROM docker.io/python:3.10

WORKDIR /

# --- [Install python and pip] ---
RUN apt-get update && apt-get upgrade -y && \
    apt-get install -y python3 python3-pip git
COPY . /

RUN pip install --no-cache-dir -r requirements.txt
RUN pip install gunicorn

ENV GUNICORN_CMD_ARGS="--workers=3 --bind=0.0.0.0:8080"

EXPOSE 8080

CMD [ "gunicorn", "main:app" ]

In [ ]:
version: '3'
services:
        web:
                image: flask_port_v1 # Change the image name to something unique to your project, aka my_unique_name_v1
                build: .
                ports:
                        - "8---:8080" # Edit the number on the left to match the port you selected
                volumes:
                        - ./volumes:/volumes
                        - ./instance:/instance
                restart: unless-stopped